In [1]:
from typing import List, Optional

import fire

from llama import Dialog, Llama
import torch.distributed as dist
import torch
import os
from data_casual import output_list_train, input_list_train

In [2]:
from data_casual import instruction_data_point

In [3]:

from llama.model import ModelArgs, Transformer
from llama.tokenizer import ChatFormat, Dialog, Message, Tokenizer

In [4]:
import json
import os
import sys
import time
from pathlib import Path
from typing import List, Optional, Tuple, TypedDict

import torch
import torch.nn.functional as F
from fairscale.nn.model_parallel.initialize import (
    get_model_parallel_rank,
    initialize_model_parallel,
    model_parallel_is_initialized,
)

In [5]:
generator = Llama.build("./", "./tokenizer.model",max_seq_len= 2500,max_batch_size= 4, activation=True, activation_layer=28)

> initializing model parallel with size 1


> initializing ddp with size 1
> initializing pipeline with size 1


c:\Users\Ribbe\Coding\VSC\FU_Berlin\Actual_Work\llama3\llama\generation.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location=

Loaded in 116.72 seconds


In [6]:
ckpt_dir = "./"
tokenizer_path = "./tokenizer.model"
temperature = 0.6
top_p = 0.9
max_seq_len = 1024
max_batch_size = 4
max_gen_len= generator.model.params.max_seq_len - 1


In [7]:
def sample_top_p(probs, p):
    """
    Perform top-p (nucleus) sampling on a probability distribution.

    Args:
        probs (torch.Tensor): Probability distribution tensor.
        p (float): Probability threshold for top-p sampling.

    Returns:
        torch.Tensor: Sampled token indices.

    Note:
        Top-p sampling selects the smallest set of tokens whose cumulative probability mass
        exceeds the threshold p. The distribution is renormalized based on the selected tokens.
    """
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    mask = probs_sum - probs_sort > p
    probs_sort[mask] = 0.0
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    next_token = torch.multinomial(probs_sort, num_samples=1)
    next_token = torch.gather(probs_idx, -1, next_token)
    return next_token

In [8]:
prompt_tokens = [generator.formatter.encode_dialog_prompt(input_list_train[1142])]

In [9]:
n = len(input_list_train)
n

205734

In [10]:
neg_activation_vector_dic = {}
pos_activation_vector_dic = {}
loss = 0
for i in range(1):
    value, activation_vec_list = generator.chat_completion([input_list_train[i]],
                                       max_gen_len=max_gen_len,
                                       top_p=top_p,
                                       temperature=temperature)
    value_numerical = "1" if value[0]["generation"]["content"] == "T" else 0
    if value_numerical == output_list_train[i]:
        pos_activation_vector_dic[i] = activation_vec_list[0] 
    else :
        neg_activation_vector_dic[i] = activation_vec_list[0]
        loss += 1/len(input_list_train)


torch.Size([1, 188, 4096])
tensor([[[-0.0664,  0.1074, -0.0164,  ...,  0.3164,  0.3828,  0.1367],
         [ 0.7188,  0.0303,  0.1816,  ..., -0.1377,  0.0913, -0.1455],
         [ 0.3711,  0.4609,  0.2871,  ...,  0.2412,  0.6094, -0.5234],
         ...,
         [ 0.0449,  0.3086,  0.0752,  ...,  0.1602,  0.4258,  0.4023],
         [-0.6719,  0.5000,  0.2090,  ..., -0.0571,  0.0986, -0.1875],
         [-0.5078,  0.5781,  0.1973,  ..., -0.2012, -0.1934,  0.1367]]])
torch.Size([1, 1, 4096])
tensor([[[ 0.2012,  0.8516, -0.1660,  ...,  0.2002,  0.1211, -0.4570]]])


In [11]:
value_instruct, activation_instruct = generator.chat_completion(instruction_data_point,
                                       max_gen_len=max_gen_len,
                                       top_p=top_p,
                                       temperature=temperature)

torch.Size([1, 58, 4096])
tensor([[[-0.0664,  0.1108, -0.0151,  ...,  0.3164,  0.3770,  0.1367],
         [ 0.7227,  0.0322,  0.1895,  ..., -0.1309,  0.0898, -0.1416],
         [ 0.3770,  0.4551,  0.2871,  ...,  0.2393,  0.6016, -0.5234],
         ...,
         [-0.0483,  0.4551, -0.1113,  ..., -0.0182, -0.1689,  0.2617],
         [-0.4375,  0.2256,  0.0522,  ...,  0.0874,  0.0156, -0.1250],
         [-0.3164,  0.1138,  0.5859,  ...,  0.0186, -0.1836, -0.2109]]])
torch.Size([1, 1, 4096])
tensor([[[ 0.1426, -0.1338,  0.1758,  ...,  0.0571,  0.2949, -0.1543]]])
torch.Size([1, 1, 4096])
tensor([[[-0.4180, -0.7070,  0.5977,  ...,  0.1748, -0.3496, -0.4219]]])
torch.Size([1, 1, 4096])
tensor([[[-0.4258,  0.0898, -0.1797,  ..., -0.9492, -0.5117,  0.0664]]])
torch.Size([1, 1, 4096])
tensor([[[-0.2559,  0.0186, -0.0947,  ..., -0.1338, -0.4355,  0.3066]]])
torch.Size([1, 1, 4096])
tensor([[[-0.4297, -0.0742,  0.1787,  ..., -0.3340, -0.4043,  0.5742]]])
torch.Size([1, 1, 4096])
tensor([[[-0.3652

In [12]:
value_instruct

[{'generation': {'role': 'assistant',
   'content': "I'm ready to provide a response. What is the statement?"}}]

In [13]:
activation_instruct

[tensor([-0.2812,  0.1436,  0.1602,  ..., -0.1445,  0.0107,  0.0918]),
 tensor([ 0.1426, -0.1338,  0.1758,  ...,  0.0571,  0.2949, -0.1543]),
 tensor([-0.4180, -0.7070,  0.5977,  ...,  0.1748, -0.3496, -0.4219]),
 tensor([-0.4258,  0.0898, -0.1797,  ..., -0.9492, -0.5117,  0.0664]),
 tensor([-0.2559,  0.0186, -0.0947,  ..., -0.1338, -0.4355,  0.3066]),
 tensor([-0.4297, -0.0742,  0.1787,  ..., -0.3340, -0.4043,  0.5742]),
 tensor([-0.3652, -0.2910,  0.2695,  ..., -0.2891, -0.2734,  0.7578]),
 tensor([-0.6055, -0.1748,  0.3633,  ..., -0.5391, -0.0137,  0.1807]),
 tensor([-0.7031,  0.3164, -0.0186,  ..., -0.4023, -0.2168, -0.2139]),
 tensor([-0.3594, -0.6875,  0.2031,  ..., -0.2773, -0.7578,  0.5312]),
 tensor([-0.4609, -0.0171, -0.2734,  ..., -0.5469, -0.5938,  0.1240]),
 tensor([-0.2617,  0.0962,  0.3047,  ..., -0.7773, -0.6602, -0.1797]),
 tensor([-0.3828,  1.2578,  0.3594,  ..., -0.5625,  0.2129,  0.0674]),
 tensor([-0.0654,  0.8242,  0.1318,  ...,  0.3477, -0.2500, -0.0820])]

In [10]:
pos_vec_sum = torch.zeros([len(neg_activation_vector_dic[1])])
for ind, item in pos_activation_vector_dic.items():
    pos_vec_sum += 1/len(pos_activation_vector_dic)*item


neg_vec_sum = torch.zeros([len(neg_activation_vector_dic[1])])
for ind, item in neg_activation_vector_dic.items():
    neg_vec_sum += 1/len(neg_activation_vector_dic)*item

steering_vector = pos_vec_sum-neg_vec_sum


In [11]:
print(steering_vector)

tensor([-0.2949,  0.0142, -0.0035,  ..., -0.1094,  0.0864,  0.1484])


In [12]:
torch.save(steering_vector, "./steering_vector_layer28.pt")